# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
import re
import nltk
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from sqlalchemy import create_engine

from gensim.test.utils import datapath
from gensim import utils
import gensim.models

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import joblib

[nltk_data] Downloading package punkt to /home/tarciso/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/tarciso/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/tarciso/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')
messages_df = pd.read_sql_table(con=engine, table_name='Message')
categories_df = pd.read_sql_table(con=engine, table_name='MessageCategoryWide')
X = messages_df.message.values
Y_df = categories_df.drop('message_id', axis=1)
Y = Y_df.values
category_columns = Y_df.columns

In [3]:
X

array(['Weather update - a cold front from Cuba that could pass over Haiti',
       'Is the Hurricane over or is it not over',
       'Looking for someone but no name', ...,
       "Proshika, operating in Cox's Bazar municipality and 5 other unions, Ramu and Chokoria, assessment, 5 kg rice, 1,5 kg lentils to 700 families.",
       'Some 2,000 women protesting against the conduct of the elections were teargassed as they tried to converge on the local electoral commission offices in the southern oil city of Port Harcourt.',
       'A radical shift in thinking came about as a result of this meeting, recognizing that HIV/AIDS is at the core of the humanitarian crisis and identifying the crisis itself as a function of the HIV/AIDS pandemic.'],
      dtype=object)

In [4]:
Y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [5]:
category_columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

### 4. Split dataset
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=199)

In [7]:
X_train

array(['crudess you are the most charm and charismatic DT member .. thanks for all your videos in Santiago.. and to talk with me twice',
       'Tidal waves triggered by an earthquake off Indonesia on Sunday swept over a vast swath of coastlines, including those in India, Indonesia, Malaysia, the Maldives, Somalia, Sri Lanka and Thailand.',
       'The earthquake happened at 4h34 in the evening and for the night. ',
       ...,
       'Earthquake of the day #Haiti or Google possibly leaving China ?',
       'When the matter had been debated previously, the Non-Aligned Movement foreign ministers firmly stated that there was no right to humanitarian intervention.',
       '@DumboNYC : Oh no RT @endtwist : ConEd power station in #DUMBO is flooding ! #nopower #sandy'],
      dtype=object)

In [8]:
y_train

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [9]:
print(X_train.shape, y_train.shape)

(19608,) (19608, 36)


### 2. Write a tokenization function to process your text data

In [10]:
def tokenize(text):
    tokens = nltk.tokenize.word_tokenize(re.sub(r"[^a-zA-Z0-9]", " ", text.lower().strip()))
    lemmatizer = WordNetLemmatizer()
    clean_tokens = [lemmatizer.lemmatize(tok) for tok in tokens if tok not in stopwords.words("english")]

    return clean_tokens

In [11]:
def tokenize_str(text):
    tokens = nltk.tokenize.word_tokenize(re.sub(r"[^a-zA-Z0-9]", " ", text.lower().strip()))
    lemmatizer = WordNetLemmatizer()
    clean_tokens = [lemmatizer.lemmatize(tok) for tok in tokens if tok not in stopwords.words("english")]
    #Return tokens list as a string joined by whitespaces
    clean_tokens_str = ' '.join(clean_tokens)

    return clean_tokens_str

In [12]:
tokenize(X[0])

['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']

In [13]:
tokenize_str(X[0])

'weather update cold front cuba could pas haiti'

### 3. Generate tokenized messages table for further analysis

In [14]:
messages_df['tokens'] = messages_df.message.apply(lambda x: tokenize_str(x))

In [15]:
messages_df

,message_id,message,original,genre,num_words,tokens
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,13,weather update cold front cuba could pas haiti
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,9,hurricane
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,6,looking someone name
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,13,un report leogane 80 90 destroyed hospital st ...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,12,say west side haiti rest country today tonight
...,...,...,...,...,...,...
26139,30261,The training demonstrated how to enhance micro...,None,news,21,training demonstrated enhance micronutrient pr...
26140,30262,A suitable candidate has been selected and OCH...,None,news,22,suitable candidate selected ocha jakarta curre...
26141,30263,"Proshika, operating in Cox's Bazar municipalit...",None,news,23,proshika operating cox bazar municipality 5 un...
26142,30264,"Some 2,000 women protesting against the conduc...",None,news,31,2 000 woman protesting conduct election tearga...


In [16]:
messages_tokens = messages_df[['message_id','tokens']]

In [17]:
messages_tokens.to_sql('MessageTokens', engine, index=False, if_exists='replace')

### 3. Find n-grams and save to DB

In [18]:
def get_ngrams_freqs(messages_array, n=1):
    vec = CountVectorizer(ngram_range=(n, n)).fit(messages_array)
    bag_of_words = vec.transform(messages_array)
    word_count = bag_of_words.sum(axis=0)
    words_freq = [(word, n, word_count[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key = lambda x: x[2], reverse=True)
    words_freq_df = pd.DataFrame(data = words_freq, columns = ['ngram','n','count'])
    return words_freq_df

In [19]:
unigrams_freqs = get_ngrams_freqs(messages_tokens.tokens)
bigrams_freqs = get_ngrams_freqs(messages_tokens.tokens, n=2)
trigrams_freqs = get_ngrams_freqs(messages_tokens.tokens, n=3)
ngrams_freqs = pd.concat([unigrams_freqs, bigrams_freqs, trigrams_freqs])

In [20]:
ngrams_freqs

,ngram,n,count
0,water,1,3034
1,people,1,2998
2,food,1,2892
3,help,1,2649
4,need,1,2484
...,...,...,...
289130,crisis identifying crisis,3,1
289131,identifying crisis function,3,1
289132,crisis function hiv,3,1
289133,function hiv aid,3,1


In [21]:
ngrams_freqs.to_sql('NGramsFreqs', engine, index=False, if_exists='replace')

### 4. Train a Word2Vec model using messages text

In [22]:
class Message(object):
    """An interator that yields messages tokens (lists of str)."""
    
    def __init__(self, messages_df, sample_size=-1):
        self.messages_df = messages_df
        self.sample_size = sample_size

    def __iter__(self):
        messages = messages_df
        
        #Sample dataset if specified
        if self.sample_size > 0:
            messages = messages.sample(self.sample_size)
        
        #Yeld tokenized message joined by whitespaces
        for token_str in messages.tokens:
            yield token_str.split()

In [23]:
messages_df.tokens

0           weather update cold front cuba could pas haiti
1                                                hurricane
2                                     looking someone name
3        un report leogane 80 90 destroyed hospital st ...
4           say west side haiti rest country today tonight
                               ...                        
26139    training demonstrated enhance micronutrient pr...
26140    suitable candidate selected ocha jakarta curre...
26141    proshika operating cox bazar municipality 5 un...
26142    2 000 woman protesting conduct election tearga...
26143    radical shift thinking came result meeting rec...
Name: tokens, Length: 26144, dtype: object

In [25]:
sentences = Message(messages_df=messages_df)
w2v_model_full = gensim.models.Word2Vec(sentences=sentences)
w2v_model_full.save("messages-word2vec-full.model")

In [26]:
w2v_model_full = gensim.models.Word2Vec.load("messages-word2vec-full.model")

In [27]:
#?w2v_model_full

In [28]:
for i, word in enumerate(w2v_model_full.wv.vocab):
    if i == 10:
        break
    print(word)

weather
update
cold
front
cuba
could
pas
haiti
hurricane
looking


In [29]:
w2v_model_full.wv['earthquake']

array([-1.4538959e+00, -1.4527936e-01,  1.0307724e+00, -5.6298780e-01,
       -3.3289680e-01,  2.8916386e-01, -7.7981776e-01, -5.1895010e-01,
        3.2228515e-01, -3.9366740e-01, -5.1296234e-01, -3.5168293e-01,
       -4.4185236e-01, -1.3428342e-01, -1.7327510e-01,  1.1468548e-02,
       -5.7652391e-02,  1.4077564e+00, -6.8507677e-01, -4.3553764e-01,
        1.3947522e-02,  7.8041744e-01,  1.7920560e-01,  5.5988103e-02,
        5.0691932e-01, -8.2600492e-01, -1.6676201e-01,  1.3172260e-01,
        6.7990023e-01, -5.6991175e-02,  1.5688449e-01,  6.9112158e-01,
       -1.1897253e-01,  3.4465411e-01,  2.0699615e-02, -1.8806231e-01,
        3.6496359e-01,  6.1426562e-01, -6.3016260e-01,  1.2678260e-01,
        1.5225254e-01,  9.6958382e-03, -7.1998346e-01, -9.6736991e-01,
       -2.1439807e-01,  1.6827160e-01,  1.1810173e+00,  9.5470929e-01,
        1.7846367e-01,  1.4244514e+00,  4.7275610e-03, -2.2476405e-01,
       -8.7652326e-01, -2.1297190e-01,  1.7765622e-01,  4.9630567e-01,
      

##### Implement Vector Aggregator

In [30]:
np.array([1,2])[0]

1

In [31]:
len(X)

26144

In [32]:
X[0]

'Weather update - a cold front from Cuba that could pass over Haiti'

In [33]:
X.shape[0]

26144

In [34]:
from collections import defaultdict

class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec_model):
        self.word2vec_model = word2vec_model
        self.num_dims = word2vec_model.wv.vector_size
            
    def fit(self, X, y):
        return self 

    def transform(self, X):
        mean_embeddings = np.empty([X.shape[0],self.num_dims])
        
        for i in range(X.shape[0]):
            doc_tokens = X[i]
            
            words_vectors_concat = [self.word2vec_model.wv[w] for w in doc_tokens if w in self.word2vec_model.wv]

            if (len(words_vectors_concat) == 0):
                words_vectors_concat = [np.zeros(self.num_dims)]
                
            #print(np.mean(words_vectors_concat, axis=0))
                
            mean_embeddings[i] = np.mean(words_vectors_concat, axis=0)
            
        return mean_embeddings
    
class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec_model):
        self.word2vec_model = word2vec_model
        self.num_dims = word2vec_model.wv.vector_size
        self.word_weights = None
        
    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        max_idf = max(tfidf.idf_)
        
        tfidf_weights = [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()]
        self.word_weights = defaultdict(lambda: max_idf, tfidf_weights)
    
        return self
    
    def transform(self, X):
        mean_embeddings = np.empty([X.shape[0],self.num_dims])
        
        for i in range(X.shape[0]):
            doc_tokens = X[i]
            
            words_vectors_concat = [self.word2vec_model.wv[w]*self.word_weights[w] for w in doc_tokens if w in self.word2vec_model.wv]

            if (len(words_vectors_concat) == 0):
                words_vectors_concat = [np.zeros(self.num_dims)]
                
            #print(np.mean(words_vectors_concat, axis=0))
                
            mean_embeddings[i] = np.mean(words_vectors_concat, axis=0)
            
        return mean_embeddings

In [35]:
test_dict = defaultdict(lambda: 1, [("Hello" , 7), ("hi" , 10), ("there" , 45),("at" , 23),("this" , 77)])
test_dict['Hi']

1

##### Use a Pipeline to train the Word2Vec estimator

In [36]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB

w2v_mean_pip = Pipeline([
    ('w2v_mean', TfidfEmbeddingVectorizer(w2v_model_full)),
    ('clf', MultiOutputClassifier(GaussianNB()))
])

In [37]:
%%time

w2v_mean_pip.fit(X_train, y_train)

CPU times: user 12.8 s, sys: 14.5 ms, total: 12.8 s
Wall time: 12.8 s


Pipeline(memory=None,
         steps=[('w2v_mean',
                 <__main__.TfidfEmbeddingVectorizer object at 0x7f41ffb17cd0>),
                ('clf',
                 MultiOutputClassifier(estimator=GaussianNB(priors=None,
                                                            var_smoothing=1e-09),
                                       n_jobs=None))],
         verbose=False)

In [38]:
%%time

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
y_pred = w2v_mean_pip.predict(X_test)

#print('accuracy %s' % accuracy_score(y_pred, y_test))
#print(classification_report(y_test, y_pred,target_names=category_columns))
for category_idx in range(y_pred.shape[1]):
    print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[category_columns[category_idx] + '-0',category_columns[category_idx] + '-1']))

              precision    recall  f1-score   support

   related-0       0.26      0.77      0.38      1532
   related-1       0.83      0.26      0.40      4950

   micro avg       0.40      0.38      0.39      6482
   macro avg       0.54      0.52      0.39      6482
weighted avg       0.69      0.38      0.39      6482

              precision    recall  f1-score   support

   request-0       0.83      0.93      0.88      5393
   request-1       0.21      0.08      0.12      1143

    accuracy                           0.79      6536
   macro avg       0.52      0.51      0.50      6536
weighted avg       0.72      0.79      0.74      6536

              precision    recall  f1-score   support

     offer-0       1.00      0.15      0.26      6506
     offer-1       0.01      1.00      0.01        30

    accuracy                           0.15      6536
   macro avg       0.50      0.57      0.13      6536
weighted avg       1.00      0.15      0.26      6536

               prec

/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

 other_aid-0       0.86      0.97      0.91      5660
 other_aid-1       0.10      0.02      0.04       876

    accuracy                           0.84      6536
   macro avg       0.48      0.50      0.47      6536
weighted avg       0.76      0.84      0.80      6536

                          precision    recall  f1-score   support

infrastructure_related-0       0.95      0.12      0.22      6111
infrastructure_related-1       0.07      0.90      0.12       425

                accuracy                           0.18      6536
               macro avg       0.51      0.51      0.17      6536
            weighted avg       0.89      0.18      0.21      6536

              precision    recall  f1-score   support

 transport-0       0.97      0.10      0.19      6219
 transport-1       0.05      0.94      0.10       317

    accuracy                           0.15      6536
   macro avg       0.51      0.52      0.14      6536
we

### 5. Create Estimator to both train and average w2v on corpus data

In [49]:
from gensim.sklearn_api import W2VTransformer
from gensim.test.utils import common_texts

model = W2VTransformer(size=10, min_count=1, seed=1)

model.fit(common_texts)
          
model.transform(['graph', 'system'])

array([[-0.02431687, -0.03657118,  0.00427647,  0.00029709,  0.01564141,
        -0.01097775, -0.01471024, -0.00328241,  0.03286636, -0.03961874],
       [ 0.02330875,  0.03287221, -0.04318658,  0.04499528, -0.03425504,
         0.0489339 ,  0.02213166, -0.00622595,  0.01583076,  0.02139627]],
      dtype=float32)

In [63]:
common_texts

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]

In [84]:
messages_df.head(5).tokens.str.split().values

array([list(['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']),
       list(['hurricane']), list(['looking', 'someone', 'name']),
       list(['un', 'report', 'leogane', '80', '90', 'destroyed', 'hospital', 'st', 'croix', 'functioning', 'need', 'supply', 'desperately']),
       list(['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight'])],
      dtype=object)

In [73]:
messages_df.head(5).tokens

0       weather update cold front cuba could pas haiti
1                                            hurricane
2                                 looking someone name
3    un report leogane 80 90 destroyed hospital st ...
4       say west side haiti rest country today tonight
Name: tokens, dtype: object

In [86]:
w2v_model = W2VTransformer()
w2v_model.fit(messages_df.tokens.str.split().values)

W2VTransformer(alpha=0.025, batch_words=10000, cbow_mean=1,
               hashfxn=<built-in function hash>, hs=0, iter=5,
               max_vocab_size=None, min_alpha=0.0001, min_count=5, negative=5,
               null_word=0, sample=0.001, seed=1, sg=0, size=100,
               sorted_vocab=1, trim_rule=None, window=5, workers=3)

In [57]:
for i, word in enumerate(w2v_model.wv.vocab):
    if i == 10:
        break
    print(word)

AttributeError: 'W2VTransformer' object has no attribute 'wv'

In [58]:
?w2v_model

'weather'

In [96]:
word = messages_df.tokens[0].split()[1]
print(word)
w2v_model.transform(word)

update


array([[-0.3032465 ,  0.05649086, -0.02101794,  0.1059568 , -0.13907477,
        -0.02215241, -0.23189439,  0.07683252, -0.02488688, -0.2979106 ,
        -0.20104954, -0.16686994, -0.10283515,  0.28013214,  0.02815698,
         0.00210302, -0.07579937,  0.26622188, -0.12642209, -0.03175807,
        -0.26515093,  0.06311958,  0.0051128 , -0.13482447,  0.13944958,
        -0.12294882,  0.14286467, -0.12234699, -0.01384199, -0.13243225,
         0.2502693 ,  0.15804473,  0.17138834,  0.5557999 , -0.00583198,
        -0.01100349,  0.09449274,  0.29267606, -0.3103327 , -0.08663655,
         0.05384598, -0.03332878, -0.20938395, -0.46939597,  0.0603793 ,
         0.19812031,  0.49147815,  0.16276427,  0.29228696, -0.01665707,
         0.20514911, -0.03787076, -0.06055886,  0.15123473, -0.02828476,
         0.2234053 ,  0.02832518, -0.08257548,  0.1880974 , -0.22986291,
        -0.00165175,  0.10911858,  0.16529597,  0.1641674 , -0.10160564,
        -0.17416325, -0.07240331,  0.24838981, -0.5

In [97]:
test_model = gensim.models.Word2Vec(messages_df.tokens.str.split().values)

In [99]:
for i, word in enumerate(test_model.wv.vocab):
    if i == 10:
        break
    print(word)

weather
update
cold
front
cuba
could
pas
haiti
hurricane
looking


In [100]:
test_model.wv['earthquake']

array([-1.478905  , -0.15925677,  0.9755623 , -0.5301034 , -0.398483  ,
        0.34562472, -0.7862477 , -0.48201653,  0.25195122, -0.49937314,
       -0.43746674, -0.3643866 , -0.478164  , -0.06343155, -0.13655871,
       -0.08980494, -0.13098012,  1.4481571 , -0.7227638 , -0.38324717,
       -0.12971745,  0.7293678 ,  0.16595615,  0.06682462,  0.53837305,
       -0.6818977 , -0.11609872,  0.16291414,  0.6442405 , -0.06131542,
        0.16638176,  0.6893078 , -0.19265199,  0.3790412 ,  0.06874911,
       -0.21933101,  0.38973883,  0.71781933, -0.6816055 ,  0.12813991,
        0.13905875,  0.05292251, -0.66648036, -1.1612517 , -0.23124596,
        0.11975669,  1.0843347 ,  0.97352123,  0.20556776,  1.3790338 ,
       -0.00524824, -0.16792603, -0.90305793, -0.23383601,  0.1718707 ,
        0.43943894, -0.45487282, -0.59823966,  1.2323052 , -0.31664556,
       -0.14899781,  0.63624203,  0.29014218,  0.34619534, -0.19817878,
       -0.38937512,  0.15323986,  0.36593965, -1.3187408 , -0.28

In [118]:
class MeanEmbeddingTrainVectorizer(object):

    def __init__(self, word2vec_model=None):
        if word2vec_model is None:
            self.word2vec_model = None
            self.num_dims = -1
        else:
            self.word2vec_model = word2vec_model
            self.num_dims = word2vec_model.wv.vector_size
            
    def fit(self, X, y):
        if self.word2vec_model is None:
            self.word2vec_model = gensim.models.Word2Vec(X)
            self.num_dims = self.word2vec_model.wv.vector_size
            
        return self 

    def transform(self, X):
        mean_embeddings = np.empty([X.shape[0],self.num_dims])
        
        for i in range(X.shape[0]):
            doc_tokens = X[i]
            
            words_vectors_concat = [self.word2vec_model.wv[w] for w in doc_tokens if w in self.word2vec_model.wv]

            if (len(words_vectors_concat) == 0):
                words_vectors_concat = [np.zeros(self.num_dims)]
                
            #print(np.mean(words_vectors_concat, axis=0))
                
            mean_embeddings[i] = np.mean(words_vectors_concat, axis=0)
            
        return mean_embeddings

In [119]:
class TfidfEmbeddingTrainVectorizer(object):
    
    def __init__(self, word2vec_model=None):
        if word2vec_model is None:
            self.word2vec_model = None
            self.num_dims = -1
        else:
            self.word2vec_model = word2vec_model
            self.num_dims = word2vec_model.wv.vector_size
        self.word_weights = None
        
        
    def fit(self, X, y):
        if self.word2vec_model is None:
            self.word2vec_model = gensim.models.Word2Vec(X)
            self.num_dims = self.word2vec_model.wv.vector_size
            
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        max_idf = max(tfidf.idf_)
        
        tfidf_weights = [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()]
        self.word_weights = defaultdict(lambda: max_idf, tfidf_weights)
    
        return self
    
    def transform(self, X):
        mean_embeddings = np.empty([X.shape[0],self.num_dims])
        
        for i in range(X.shape[0]):
            doc_tokens = X[i]
            
            words_vectors_concat = [self.word2vec_model.wv[w]*self.word_weights[w] for w in doc_tokens if w in self.word2vec_model.wv]

            if (len(words_vectors_concat) == 0):
                words_vectors_concat = [np.zeros(self.num_dims)]
                
            #print(np.mean(words_vectors_concat, axis=0))
                
            mean_embeddings[i] = np.mean(words_vectors_concat, axis=0)
            
        return mean_embeddings

In [123]:
%%time

from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report

w2v_mean_pip = Pipeline([
    ('w2v_tfidf', TfidfEmbeddingTrainVectorizer()),
    ('clf', MultiOutputClassifier(GaussianNB()))
])

w2v_mean_pip.fit(X_train, y_train)

y_pred = w2v_mean_pip.predict(X_test)

for category_idx in range(y_pred.shape[1]):
    print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[category_columns[category_idx] + '-0',category_columns[category_idx] + '-1']))

              precision    recall  f1-score   support

   related-0       0.38      0.04      0.07      1532
   related-1       0.76      0.93      0.84      4950

   micro avg       0.75      0.72      0.74      6482
   macro avg       0.57      0.48      0.45      6482
weighted avg       0.67      0.72      0.66      6482

              precision    recall  f1-score   support

   request-0       0.83      0.96      0.89      5393
   request-1       0.28      0.08      0.12      1143

    accuracy                           0.80      6536
   macro avg       0.56      0.52      0.51      6536
weighted avg       0.73      0.80      0.76      6536

              precision    recall  f1-score   support

     offer-0       1.00      0.99      0.99      6506
     offer-1       0.02      0.03      0.03        30

    accuracy                           0.99      6536
   macro avg       0.51      0.51      0.51      6536
weighted avg       0.99      0.99      0.99      6536

               prec

/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                          precision    recall  f1-score   support

infrastructure_related-0       0.96      0.14      0.24      6111
infrastructure_related-1       0.07      0.91      0.13       425

                accuracy                           0.19      6536
               macro avg       0.51      0.52      0.19      6536
            weighted avg       0.90      0.19      0.24      6536

              precision    recall  f1-score   support

 transport-0       0.97      0.13      0.23      6219
 transport-1       0.05      0.93      0.10       317

    accuracy                           0.17      6536
   macro avg       0.51      0.53      0.16      6536
weighted avg       0.93      0.17      0.22      6536

              precision    recall  f1-score   support

 buildings-0       0.97      0.13      0.23      6209
 buildings-1       0.05      0.92      0.10       327

    accuracy                           0.17      6536
   macro avg       0.51      0.52      0.17      6536
we

### 5. Use Glove pre-trained model to generate feature vectors

In [39]:
# from gensim.test.utils import datapath, get_tmpfile
# from gensim.models import KeyedVectors
# from gensim.scripts.glove2word2vec import glove2word2vec

# glove_file = 'glove-pretrained/glove.6B.50d.txt'
# tmp_file = 'glove-pretrained/glove.6B.50d_word2vec.txt'

# _ = glove2word2vec(glove_file, tmp_file)

# model = KeyedVectors.load_word2vec_format(tmp_file)

In [41]:
from gensim.models import KeyedVectors

glove_w2v = KeyedVectors.load_word2vec_format('glove-pretrained/glove.6B.50d_word2vec.txt', binary=False)

##### Use a Pipeline to train the Word2Vec estimator

In [42]:
w2v_mean_pip = Pipeline([
    ('w2v_tfidf', TfidfEmbeddingVectorizer(glove_w2v)),
    ('clf', MultiOutputClassifier(GaussianNB()))
])

/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/ipykernel_launcher.py:31: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [43]:
%%time

w2v_mean_pip.fit(X_train, y_train)

/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/ipykernel_launcher.py:50: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


CPU times: user 41.8 s, sys: 5.46 ms, total: 41.8 s
Wall time: 41.8 s


Pipeline(memory=None,
         steps=[('w2v_tfidf',
                 <__main__.TfidfEmbeddingVectorizer object at 0x7f41e9ad0f90>),
                ('clf',
                 MultiOutputClassifier(estimator=GaussianNB(priors=None,
                                                            var_smoothing=1e-09),
                                       n_jobs=None))],
         verbose=False)

In [44]:
%%time

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
y_pred = w2v_mean_pip.predict(X_test)

#print('accuracy %s' % accuracy_score(y_pred, y_test))
#print(classification_report(y_test, y_pred,target_names=category_columns))
for category_idx in range(y_pred.shape[1]):
    print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[category_columns[category_idx] + '-0',category_columns[category_idx] + '-1']))

/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/ipykernel_launcher.py:50: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


              precision    recall  f1-score   support

   related-0       0.33      0.02      0.03      1532
   related-1       0.76      0.96      0.85      4950

   micro avg       0.76      0.73      0.74      6482
   macro avg       0.55      0.49      0.44      6482
weighted avg       0.66      0.73      0.65      6482

              precision    recall  f1-score   support

   request-0       0.83      0.95      0.89      5393
   request-1       0.23      0.07      0.10      1143

    accuracy                           0.80      6536
   macro avg       0.53      0.51      0.50      6536
weighted avg       0.72      0.80      0.75      6536

              precision    recall  f1-score   support

     offer-0       1.00      0.98      0.99      6506
     offer-1       0.00      0.00      0.00        30

    accuracy                           0.97      6536
   macro avg       0.50      0.49      0.49      6536
weighted avg       0.99      0.97      0.98      6536

               prec

/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                          precision    recall  f1-score   support

infrastructure_related-0       0.96      0.13      0.23      6111
infrastructure_related-1       0.07      0.92      0.13       425

                accuracy                           0.18      6536
               macro avg       0.51      0.52      0.18      6536
            weighted avg       0.90      0.18      0.22      6536

              precision    recall  f1-score   support

 transport-0       0.97      0.14      0.25      6219
 transport-1       0.05      0.93      0.10       317

    accuracy                           0.18      6536
   macro avg       0.51      0.53      0.17      6536
weighted avg       0.93      0.18      0.24      6536

              precision    recall  f1-score   support

 buildings-0       0.97      0.14      0.24      6209
 buildings-1       0.05      0.92      0.10       327

    accuracy                           0.18      6536
   macro avg       0.51      0.53      0.17      6536
we

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
pipeline = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize, ngram_range=(1,3))),
    ('tfidf', TfidfTransformer()),
    ('multi_clf', MultiOutputClassifier(RandomForestClassifier(random_state=199), n_jobs=-1))
])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [13]:
X_train.shape

(19662,)

In [14]:
X_train

array(['It works as a fertilizer enhancement, a composting additive, and even as an alternative to household cleaning products, according to EMRO.',
       'I need a job to feed my family. I am an electrician and a musician. I perform harmonization and orchestration in Mexico. I speak English and French. Let me know how I can help',
       'The report also warns that if the clan fighting extends to sorghum-producing areas in the south during this critical stage of crop harvest, it could further hamper harvesting and exert a negative impact on food security throughout Somalia.',
       ...,
       'HELP THE EARTHQUAKE VICTIMS IN HAITI http tinyurl.com yk3bspe links to many resources where you can make donations. Broke? donate time',
       'The low snowfall pattern and the snowfall differentiation among catchments are clearly reflected in the irrigated wheat production levels.',
       "Having fun at #Lexi 's ! Real food in real house with electrical lighting . Good times . I feel human

In [15]:
# train classifier
pipeline.fit(X_train, y_train)

# predict on test data
y_pred = pipeline.predict(X_test)

ValueError: WRITEBACKIFCOPY base is read-only

In [ ]:
print(X_test.shape)

In [ ]:
print(y_pred.shape)
y_pred

In [ ]:
y_pred[:,0]

In [ ]:
y_test

In [ ]:
for category_idx in range(y_pred.shape[1]):
    print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[df.columns[4+category_idx] + '-0',df.columns[4+category_idx] + '-1']))

### 6. Improve your model
Use grid search to find better parameters. 

In [20]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'multi_clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'multi_clf__estimator__bootstrap', 'multi_clf__estimator__class_weight', 'multi_clf__estimator__criterion', 'multi_clf__estimator__max_depth', 'multi_clf__estimator__max_features', 'multi_clf__estimator__max_leaf_nodes', 'multi_clf__estimator__min_impurity_decrease', 'multi_clf__estimator__min_impurity_split', 'multi_clf__estimator__min_samples_leaf', 'multi_clf__estimator__min_samples_split', 'multi_clf__estimator__min_weight_fraction_leaf', 'multi_clf__estimator__n_estimators', 'multi_clf__estimator__n_jobs', 'multi_clf__estimator

In [ ]:
parameters = {
        'multi_clf__estimator__n_estimators': [20,50],
        'multi_clf__estimator__max_depth': [50, 100]
    }

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=10)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [31]:
# train classifier
cv.fit(X_train, y_train)

# predict on test data
y_pred = cv.predict(X_test)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV]  ................................................................
[CV] ...................... , score=0.22017088800732376, total= 2.7min
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.0min remaining:    0.0s


[CV] ...................... , score=0.22978333841928594, total= 2.4min
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  5.7min remaining:    0.0s


[CV] ...................... , score=0.22642660970399756, total= 2.5min


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  8.4min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  8.4min finished


In [32]:
cv.best_params_

{}

In [33]:
for category_idx in range(y_pred.shape[1]):
    print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[df.columns[4+category_idx] + '-0',df.columns[4+category_idx] + '-1']))

             precision    recall  f1-score   support

  related-0       0.55      0.32      0.41      1565
  related-1       0.81      0.91      0.86      4941

avg / total       0.74      0.77      0.75      6506

             precision    recall  f1-score   support

  request-0       0.89      0.98      0.93      5420
  request-1       0.81      0.39      0.53      1134

avg / total       0.87      0.88      0.86      6554

             precision    recall  f1-score   support

    offer-0       1.00      1.00      1.00      6524
    offer-1       0.00      0.00      0.00        30

avg / total       0.99      1.00      0.99      6554

               precision    recall  f1-score   support

aid_related-0       0.71      0.89      0.79      3900
aid_related-1       0.75      0.46      0.57      2654

  avg / total       0.73      0.72      0.70      6554

                precision    recall  f1-score   support

medical_help-0       0.93      1.00      0.96      6038
medical_help-1     

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [122]:
# Output a pickle file for the model
joblib.dump(cv, 'classifier.pkl') 

['initial_model.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.